In [ ]:
import sys

from ot import image_processing
try:
    import ot

except ModuleNotFoundError:
    sys.path.append("../")

    from ot import interfaces, algoritmi, coreg, metodi

In [4]:
from dataclasses import dataclass

In [93]:
def __gen(n):
    for i in range(n):
        yield f"b{i+1}"

In [181]:
class Image:
    def __init__(self, *args):
        self.bandnames = tuple((f"B{i+1}" for i in range(len(args))))
        for name, arg in zip(self.bandnames, args):
            setattr(self, name, arg)
    
    def __iter__(self):
        for band in self.bandnames:
            yield self.__getattribute__(band)
        
    def __getitem__(self, __index):
        name = list(self.bandnames)[__index]
        return self.__getattribute__(name)
    
    def get_band(self, n: str | int | None = None):
        if n in self.bandnames:
            return self.__getattribute__(n)
        
        elif isinstance(n, int):
            return self[n]
        
        elif n is None:
            return self


In [184]:
import numpy as np

i = Image(*np.random.randint(0, 255, (19, 102, 234)))

for n, band in enumerate(i):
    print(n, band)

0 [[234 218 252 ... 101 248 222]
 [ 24  58   0 ...  64  20 122]
 [213 154 182 ... 139 204  63]
 ...
 [199 124 197 ... 119  92  96]
 [253 105 251 ... 217  16 190]
 [  1  63 243 ... 122 120 219]]
1 [[123 158  57 ... 209 134 164]
 [149 108  19 ... 148 237 193]
 [ 94  78  82 ...  14 193 109]
 ...
 [147 177  87 ...  74  62  86]
 [253  98 227 ... 183 241 230]
 [213 188  75 ... 252 156  67]]
2 [[ 25 130  38 ... 186 116 108]
 [ 22  49 108 ...  50   1  80]
 [246 202  51 ... 250  40 168]
 ...
 [208 128 112 ... 114 148 161]
 [  4  53 200 ... 203  90  60]
 [109 178  26 ... 122 161  15]]
3 [[180 150  43 ... 170  15 197]
 [214  84 150 ...  66 192 198]
 [157 230 205 ... 197   7 245]
 ...
 [ 30 126  79 ... 208 197 184]
 [  7 131 111 ...  56  44 226]
 [236  61 114 ... 104 243 192]]
4 [[163 135 178 ... 139 150  89]
 [ 58 211 115 ... 160 151   8]
 [216  64 106 ... 201  38  97]
 ...
 [114  79 186 ... 180 123  43]
 [120 220 185 ... 138  40 173]
 [182 129 203 ... 235  46 159]]
5 [[110 116  85 ...  92 253 19

In [60]:
from collections import namedtuple


Band = namedtuple("Band", ['RED', 'GREEN', 'BLUE'])

In [66]:
class Band(list):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [70]:
from collections import OrderedDict

In [ ]:
class Image:
    def __init__(self, *args):
        for name, arg in zip(range(999), args):
            setattr(self, str(name), arg)
    
    def __iter__(self)

In [86]:
i = Image(12, 12, 23, 34, 5)

i.__getattribute__('3')

34

In [33]:
class Algorithm:
    def __init__(self, a, b, c, d, e, f, g):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        self.f = f
        self.g = g

    @staticmethod
    def from_dict(__d: dict):
        keys = list("abcdefg")

        kw = dict()
        for key in keys:
            kw[key] = __d.get(key)
        
        return Algorithm(**kw)


class Operation:
    def __init__(self, method):
        self.method = method

    def __call__(self, **kw):
        m = self.method.from_dict(kw)
        print(f"{kw['h'] = }")

In [35]:
o = Operation(Algorithm)

keys = list("abcdefgh")
values = [*range(3, 8+3)]

kw = dict(zip(keys, values))
o(**kw)

kw['h'] = 10


In [ ]:

# Interfaccia comune per tutti gli algoritmi di offset tracking, con parametri aggiuntivi
class RasterDisplacement(ABC):

    @staticmethod
    def from_dict(__d: dict): ...

    @staticmethod
    def from_JSON(__json: Path | str):
        __d = json.loads(Path(__json).read_text())

        return RasterDisplacement.from_dict(__d)

    @staticmethod
    def from_YAML(__yaml: Path | str):
        import yaml

        __d = yaml.safe_load(Path(__yaml).read_text())

        return RasterDisplacement.from_dict(__d)

    def toJSON(self):

        try:
            parms = self.__dict__
            parms['flags'] = self.flags.value
        except AttributeError:
            parms = self.__dict__

        return json.dumps(parms, indent=4)
    
    @abstractmethod
    def run(self, *args, **kwargs):
        pass


In [56]:
import rasterio
from rasterio import Affine, transform


def write_raster(fpath: str | Path, array: np.ndarray, metadata: dict) -> None:

    if array.ndim > 2:
        nbands = array.shape[0]
    else:
        nbands = 1

    with rasterio.open(fp=str(fpath), mode='w', **metadata) as dst:
        dst.write(array, nbands)


class Raster:
    def __init__(self, array: np.ndarray,
                 metadata: dict,
                 mask: np.ndarray | None = None) -> None:
        
        self.array = array
        self.metadata = metadata
        self.mask = mask

    def write_raster(self, filename): ...

    @staticmethod
    def read_raster(self, filename, **kwargs):

        band = kwargs.get('band', 1)

        with rasterio.open(filename, 'r', **kwargs) as rds:
            array = rds.read(band).astype(np.float32)
            mask = ~rds.dataset_mask().astype(bool)
            meta = rds.meta
            meta['dtype'] = 'float32'

        if mask.ndim != array.ndim:
            mask = np.repeat(mask[np.newaxis, ...], array.ndim, axis=0)

        return Raster(array, meta, mask)
    

In [30]:
class Alg1(RasterDisplacement):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

    def from_dict(__d: dict):
        keys = [
            "a",
            "b",
            "c"
        ]

        kw = dict()
        for key in keys:
            kw[key] = __d.get(key, None)

        return Alg1(**kw)
    
    def run(self, reference, target):
        return (reference * target) * (self.a * (self.b**self.c))

In [55]:
params = dict(a=15.3,
              b=124.23, 
              c=-0.2)

reference = np.random.rand(1000, 1000)
target = np.random.rand(1000, 1000)
Alg1(**params).run(reference, target)

array([[4.26425764, 1.68417392, 0.23250655, ..., 2.27289342, 0.18301154,
        4.14604602],
       [0.28160721, 3.22186233, 0.80401931, ..., 0.09780284, 1.49502781,
        0.49083635],
       [1.62421028, 1.81317085, 3.06475166, ..., 2.09133113, 0.39998904,
        1.05289761],
       ...,
       [3.62856853, 0.11560847, 1.13257494, ..., 0.33419445, 0.10323042,
        0.40788751],
       [0.04641018, 3.88007568, 0.54648988, ..., 2.37805085, 1.06878146,
        0.31393265],
       [0.03095252, 3.00622249, 0.7329174 , ..., 3.93609606, 1.88066385,
        1.06475609]])

In [1]:
from paracelso_wp32 import s1

ModuleNotFoundError: No module named 'paracelso_wp32'